In [ ]:
import cv2
import mediapipe as mp
import math

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Reference keypoints (example for Warrior II pose)
reference_pose = {
    "left_arm_angle": 90,  # Ideal angle between left shoulder, elbow, and wrist
    "right_arm_angle": 180,  # Ideal angle for the right arm
    "left_leg_angle": 90,  # Angle between hip, knee, and ankle
}

# Helper function to calculate angle between three points
def calculate_angle(a, b, c):
    a = [a.x, a.y]
    b = [b.x, b.y]
    c = [c.x, c.y]
    angle = math.degrees(
        math.atan2(c[1] - b[1], c[0] - b[0]) - math.atan2(a[1] - b[1], a[0] - b[0])
    )
    return abs(angle)

# Open the webcam
cap = cv2.VideoCapture(1)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Pose
    result = pose.process(frame_rgb)

    # Draw keypoints on the frame
    if result.pose_landmarks:
        mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Extract keypoints
        landmarks = result.pose_landmarks.landmark

        # Calculate angles for feedback
        left_arm_angle = calculate_angle(
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER],
            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW],
            landmarks[mp_pose.PoseLandmark.LEFT_WRIST],
        )
        right_arm_angle = calculate_angle(
            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER],
            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW],
            landmarks[mp_pose.PoseLandmark.RIGHT_WRIST],
        )
        left_leg_angle = calculate_angle(
            landmarks[mp_pose.PoseLandmark.LEFT_HIP],
            landmarks[mp_pose.PoseLandmark.LEFT_KNEE],
            landmarks[mp_pose.PoseLandmark.LEFT_ANKLE],
        )

        # Compare with reference pose
        feedback = []
        if abs(left_arm_angle - reference_pose["left_arm_angle"]) > 10:
            feedback.append("Raise your left arm to 90 degrees.")
        if abs(right_arm_angle - reference_pose["right_arm_angle"]) > 10:
            feedback.append("Straighten your right arm.")
        if abs(left_leg_angle - reference_pose["left_leg_angle"]) > 10:
            feedback.append("Bend your left knee more.")

        # Display feedback on the frame
        for i, text in enumerate(feedback):
            cv2.putText(frame, text, (10, 30 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Display the frame
    cv2.imshow("Yoga Pose Corrector", frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1732289807.267742 2677061 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1732289807.353243 2677153 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732289807.367492 2677153 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732289809.224850 2677152 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


KeyboardInterrupt: 

: 